# Imports

In [10]:
import os
import time
import torch
import torchaudio
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts

In [11]:
print("Loading model...")
config = XttsConfig()
config

Loading model...


XttsConfig(output_path='output', logger_uri=None, run_name='run', project_name=None, run_description='🐸Coqui trainer run.', print_step=25, plot_step=100, model_param_stats=False, wandb_entity=None, dashboard_logger='tensorboard', save_on_interrupt=True, log_model_step=None, save_step=10000, save_n_checkpoints=5, save_checkpoints=True, save_all_best=False, save_best_after=0, target_loss=None, print_eval=False, test_delay_epochs=0, run_eval=True, run_eval_steps=None, distributed_backend='nccl', distributed_url='tcp://localhost:54321', mixed_precision=False, precision='fp16', epochs=1000, batch_size=32, eval_batch_size=16, grad_clip=0.0, scheduler_after_epoch=True, lr=0.001, optimizer='radam', optimizer_params=None, lr_scheduler=None, lr_scheduler_params={}, use_grad_scaler=False, allow_tf32=False, cudnn_enable=True, cudnn_deterministic=False, cudnn_benchmark=False, training_seed=54321, model='xtts', num_loader_workers=0, num_eval_loader_workers=0, use_noise_augment=False, audio=XttsAudio

In [12]:
#config.load_json("config.json")


In [13]:
model = Xtts.init_from_config(config)
model

Xtts(
  (gpt): GPT(
    (conditioning_encoder): ConditioningEncoder(
      (init): Conv1d(80, 1024, kernel_size=(1,), stride=(1,))
      (attn): Sequential(
        (0): AttentionBlock(
          (norm): GroupNorm32(32, 1024, eps=1e-05, affine=True)
          (qkv): Conv1d(1024, 3072, kernel_size=(1,), stride=(1,))
          (attention): QKVAttention()
          (x_proj): Identity()
          (proj_out): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,))
        )
        (1): AttentionBlock(
          (norm): GroupNorm32(32, 1024, eps=1e-05, affine=True)
          (qkv): Conv1d(1024, 3072, kernel_size=(1,), stride=(1,))
          (attention): QKVAttention()
          (x_proj): Identity()
          (proj_out): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,))
        )
        (2): AttentionBlock(
          (norm): GroupNorm32(32, 1024, eps=1e-05, affine=True)
          (qkv): Conv1d(1024, 3072, kernel_size=(1,), stride=(1,))
          (attention): QKVAttention()
          (x_proj): Ide

In [14]:
model.load_checkpoint(config, checkpoint_dir="/home/jack/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v2", use_deepspeed=False)


In [15]:
model.cuda()

Xtts(
  (gpt): GPT(
    (conditioning_encoder): ConditioningEncoder(
      (init): Conv1d(80, 1024, kernel_size=(1,), stride=(1,))
      (attn): Sequential(
        (0): AttentionBlock(
          (norm): GroupNorm32(32, 1024, eps=1e-05, affine=True)
          (qkv): Conv1d(1024, 3072, kernel_size=(1,), stride=(1,))
          (attention): QKVAttention()
          (x_proj): Identity()
          (proj_out): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,))
        )
        (1): AttentionBlock(
          (norm): GroupNorm32(32, 1024, eps=1e-05, affine=True)
          (qkv): Conv1d(1024, 3072, kernel_size=(1,), stride=(1,))
          (attention): QKVAttention()
          (x_proj): Identity()
          (proj_out): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,))
        )
        (2): AttentionBlock(
          (norm): GroupNorm32(32, 1024, eps=1e-05, affine=True)
          (qkv): Conv1d(1024, 3072, kernel_size=(1,), stride=(1,))
          (attention): QKVAttention()
          (x_proj): Ide

In [16]:

print("Computing speaker latents...")
gpt_cond_latent, speaker_embedding = model.get_conditioning_latents(audio_path=["latent_GSH.wav"])
gpt_cond_latent, speaker_embedding

Computing speaker latents...


(tensor([[[ 0.4033,  0.1694,  1.1336,  ...,  0.3980, -0.6568,  0.3760],
          [ 0.2801, -0.8360,  1.5940,  ...,  0.7739, -0.5652, -0.0047],
          [-0.1769,  0.1562, -0.6511,  ...,  0.4006, -1.3897, -0.1062],
          ...,
          [ 0.2139,  0.0228, -0.1127,  ...,  0.5733, -0.2708, -0.1911],
          [-0.4854,  1.0494,  1.1974,  ..., -0.7586, -0.2296,  0.1447],
          [ 0.2372, -0.0682,  1.5510,  ...,  0.9232, -1.2930,  0.1247]]],
        device='cuda:0'),
 tensor([[[-2.6305e-02],
          [-2.7087e-02],
          [ 2.1665e-02],
          [-5.4963e-02],
          [ 5.8202e-02],
          [-4.9175e-02],
          [-1.6467e-02],
          [-3.8831e-02],
          [ 4.5130e-02],
          [-2.0003e-02],
          [ 3.3590e-02],
          [ 1.2272e-02],
          [ 4.2224e-02],
          [-2.6710e-02],
          [-2.0062e-02],
          [-2.1825e-03],
          [-1.1786e-01],
          [-6.5712e-03],
          [-1.6502e-02],
          [ 1.5121e-02],
          [-1.1136e-02],


In [17]:
print("Inference...")
t0 = time.time()
chunks = model.inference_stream(
        """
The revolution will not be right back after a message about a white tornado white lightning or white people,
The revolution will not go better with Coke,
The revolution will be no re-run brothers,
The revolution will be live""",
    "en",
    gpt_cond_latent,
    speaker_embedding
)


Inference...


In [21]:
chunks

<generator object Xtts.inference_stream at 0x7f79d2181460>

In [18]:

wav_chuncks = []
for i, chunk in enumerate(chunks):
    if i == 0:
        print(f"Time to first chunck: {time.time() - t0}")
    print(f"Received chunk {i} of audio length {chunk.shape[-1]}")
    wav_chuncks.append(chunk)
wav = torch.cat(wav_chuncks, dim=0)
torchaudio.save("xtts_streaming.wav", wav.squeeze().unsqueeze(0).cpu(), 24000)


/home/jack/.pyenv/versions/ttsengine/lib/python3.10/site-packages/TTS/tts/layers/xtts/stream_generator.py:138: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


TypeError: isin() received an invalid combination of arguments - got (test_elements=int, elements=Tensor, ), but expected one of:
 * (Tensor elements, Tensor test_elements, *, bool assume_unique = False, bool invert = False, Tensor out = None)
 * (Number element, Tensor test_elements, *, bool assume_unique = False, bool invert = False, Tensor out = None)
 * (Tensor elements, Number test_element, *, bool assume_unique = False, bool invert = False, Tensor out = None)
